## 3. Эффективные функции
### 3.1 Функции Python - это объекты первого класса
Функции в python можно присваивать переменным, хранить их в структурах данных, передавать их в качестве аргументов другим функциям и даже возвращать их в качестве значений из других функций

In [16]:
def yell(text):
    return text.upper() + '!'

yell('привет')

'ПРИВЕТ!'

Функции - тоже объекты. Объекты-функции и их имена - два отдельных элемента

In [17]:
bark = yell # теперь bark тоже указывает на функцию, ту же, что и yell
bark('гав')

del yell
bark('эй') # даже после удаления оригинальной ссылки, можно вызвать функцию через другое имя

bark.__name__ # атрибут __name__ до сих пор показывает yell

'yell'

Функции могут храниться в структурах данных

In [18]:
funcs = [bark, str.lower, str.capitalize]
funcs 

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [19]:
for f in funcs:
    print(f, f('привет')) 

funcs[0]('гав') # вызов функции из списка

<function yell at 0x0000020225B31B20> ПРИВЕТ!
<method 'lower' of 'str' objects> привет
<method 'capitalize' of 'str' objects> Привет


'ГАВ!'

Функции могут передаваться другим функциям

In [20]:
def greet(func):
    greeting = func('Привет! Я - программа Python')
    print(greeting)

greet(bark) # передача функции в другую функцию

def whisper(text):
    return text.lower() + '...'

greet(whisper)

ПРИВЕТ! Я - ПРОГРАММА PYTHON!
привет! я - программа python...


Функции, которые в качестве аргументов могут принимать другие функции, называются функциями более высокого порядка

In [21]:
list(map(bark, ['гав', 'гав-гав', 'гав-гав-гав'])) 

['ГАВ!', 'ГАВ-ГАВ!', 'ГАВ-ГАВ-ГАВ!']

Вложенные функции

In [22]:
def speak(text):
    def whisper(t):                 # вложенная функция whisper, которая видна только внутри speak
        return t.lower() + '...'
    return whisper(text)

speak('Привет!') 

'привет!...'

In [ ]:
whisper ('Привет!') # ошибка, whisper не видна вне speak
speak.whisper('Привет!') # ошибка, whisper не видна вне speak

In [ ]:
def get_speak_func(volume):
    def whisper(t):                
        return t.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

get_speak_func(0.3)
get_speak_func(0.7)

speak_func = get_speak_func(0.7)
speak_func('Привет!') # вызов функции, возвращенной другой функцией

'ПРИВЕТ!!'

#### Функции могут захватывать локальное состояние
Внутренние функции получают доступ к параметру, определенному в родительской функции. Функции, которые это делают, называются лексическими замыканиями lexical closures. Это означает, что функции могут не только возвращать линии поведения, но и предварительно их конфигурировать

In [ ]:
def get_speak_func(text, volume):
    def whisper():                
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper
    
get_speak_func('Привет, мир!', 0.7)()

'ПРИВЕТ, МИР!!'

In [ ]:
def make_adder(n):
    def add(x):
        return x + n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)
plus_3(4)
plus_5(4)

9

#### Объекты могут вести себя как функции
Все функции являются объектами, но не все объекты являются функциями, однако их можно сделать вызываемыми

In [ ]:
class Adder:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        return self.n + x
    
plus_3 = Adder(3)
plus_3(4)

7

In [ ]:
callable(plus_3)    # True
callable(yell)      # True
callable('Hello')   # False

### 3.2 Лямбды - это функции одного выражения
Для лямбда функции не приходится связывать объект функцию с именем

In [ ]:
add = lambda x, y: x + y
add(3, 5)

8

In [ ]:
def add(x, y):
    return x + y
add(3, 5)

8

In [ ]:
(lambda x, y: x + y)(5, 3)

8

#### Лямбды в вашем распоряжении

In [ ]:
tuples = [(1, 'd'), (2, 'b'), (3, 'c'), (4, 'a')]
sorted(tuples, key=lambda x: x[1])  # сортировка по второму элементу кортежа

# operator.itemgetter()

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [ ]:
sorted(range(-5, 6), key=lambda x: x * x) # сортировка по квадрату числа

# abs()

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

Лямбды работают также, как лексические замыкания - функции, которые помнят значения из объемлющего лексического контекста, даже когда поток управления программы больше не находится в этом контексте. В примере ниже лямбда по прежнему может получать доступ к значению n, несмотря на то, что она была определена в make_adder (объемлющем контексте)

In [ ]:
def make_adder(n):
    return lambda x: x + n

plus_3 = make_adder(3)
plus_5 = make_adder(5)
plus_3(4)

7

#### А может не надо

In [ ]:
# Вредно
list(filter(lambda x: x % 2 == 0, range(16)))

# Лучше
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

### 3.3 Сила декораторов
Декораторы позволяют расширять поведение вызываемых объектов без необратимой модификации самих вызываемых объектов

#### Основы декораторов в Python
Синтаксис @ декорирует функцию непосредственно во время ее определения. Чтобы получить доступ к недекорированному оригиналу, можно декорировать некоторые функции вручную

In [ ]:
# Самый простой декоратор
def null_decorator(func):
    return func

def greet():
    return 'Hello!'

greet = null_decorator(greet) # декорирование вручную

greet()


'Hello!'

In [ ]:
def null_decorator(func):
    return func

@null_decorator
def greet():
    return 'Hello!' # полностью декорированная функция

greet()

'Hello!'

#### Декораторы могут менять поведение

In [ ]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def greet():
    return 'Hello!'

greet()

'HELLO!'

In [ ]:
greet
null_decorator(greet)
uppercase(greet)        # возвращает другой объект-функцию

<function __main__.uppercase.<locals>.wrapper()>

#### Применение многочисленных декораторов к функции
Декораторы применяются снизу вверх

In [ ]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

@strong
@emphasis
def greet():
    return 'Hello!'

greet()

decorated_greet = strong(emphasis(greet))  # эквивалентно @strong @emphasis

'<strong><em>Hello!</em></strong>'

#### Декорирование функций, принимающих аргументы
В определении замыкания wrapper он использует операторы * и **, чтобы собрать все позиционные и именованные аргументы и помещает их в переменные args и kwargs

Замыкание затем переадресует собранные аргументы в оригинальную входную функциию, используя операторы распаковки аргументов * и **

In [ ]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

In [ ]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func.__name__}() with args: {args}, kwargs: {kwargs}')
        original_result = func(*args, **kwargs)
        print(f'{func.__name__}() returned: {original_result}')
        return original_result
    return wrapper

@trace
def say(name, line):
    return f'{name} says: {line}'

say('Alice', 'Hello!')

Calling say() with args: ('Alice', 'Hello!'), kwargs: {}
say() returned: Alice says: Hello!


'Alice says: Hello!'

#### Как писать "отлаживаемые" декораторы
При попытке получить доступ к метаданным функции, выдаются метаданные замыкания-обертки

In [ ]:
def greet():
    '''Вернуть приветствие'''

decorated_greet = uppercase(greet)  # декорирование функции greet
greet.__name__  # 'greet'
greet.__doc__   # 'Вернуть приветствие'
decorated_greet.__name__  # 'wrapper'
decorated_greet.__doc__  # None, метаданные потеряны

In [ ]:
import functools
def uppercase(func):
    @functools.wraps(func)  # сохраняем метаданные оригинальной функции
    def wrapper():
        return func().upper()
    return wrapper

@uppercase
def greet():
    '''Вернуть приветствие'''
    return 'Hello!'

greet.__name__  # 'greet'
greet.__doc__   # 'Вернуть приветствие'

'Вернуть приветствие'

### 3.4 Веселье с *args и **kwargs
Даные параметры позволяют функции принимать необязательные документы

Их можно назвать как угодно (если есть * и **), но принято называть args, kwargs

In [ ]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)    # выводит кортеж с позиционными аргументами
    if kwargs:
        print(kwargs)  # выводит словарь с именованными аргументами

# foo() # ошибка, требуется хотя бы один обязательный аргумент
foo('Hello')                                            # передача только обязательного аргумента
foo('Hello', 1, 2, 3)                                   # передача позиционных аргументов
foo('Hello', 1, 2, 3, key1='value1', key2='value2')     # передача именованных аргументов

Hello
Hello
(1, 2, 3)
Hello
(1, 2, 3)
{'key1': 'value1', 'key2': 'value2'}


#### Переадресация необязательных или именованных аргументов

In [ ]:
def foo(x, *args, **kwargs):
    kwargs['имя'] = 'Алиса'
    new_args = args + ('дополнительный', )
    bar(x, *new_args, **kwargs)  # передача аргументов в другую функцию

class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'синий'

AlwaysBlueCar('красный', 10000).color  # цвет будет 'синий', несмотря на переданный аргумент

'синий'

In [25]:
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs)
        print(result)
    return decorated_function

@trace
def greet(greeting, name):
    return '{}, {}!'.format(greeting, name)

greet('Привет', 'Мир')  

<function greet at 0x0000020225BD6CA0> ('Привет', 'Мир') {}
Привет, Мир!


### 3.5 Распаковка аргументов функции
Размещение * перед итерируемым объектом в вызове функции его распакует и передаст его элементы как отдельные позиционные аргументы в вызванную функцию

In [31]:
def print_vector(x, y, z):
    print('<%s, %s, %s>' % (x, y, z))

print_vector(1, 2, 3)  

tuple_vec = (1, 2, 3)
list_vec = [1, 2, 3]
print_vector(tuple_vec[0], # распаковка кортежа вручную
             tuple_vec[1], 
             tuple_vec[2])  
print_vector(*tuple_vec)  # распаковка кортежа с помощью *
print_vector(*list_vec)  # распаковка списка с помощью *

genexpr = (x * x for x in range(3))
print_vector(*genexpr)  # распаковка генератора с помощью *

dict_vec = {'x': 1, 'y': 2, 'z': 3}
print_vector(**dict_vec)  # распаковка словаря с помощью ** 
print_vector(*dict_vec)   # передача только ключей словаря

<1, 2, 3>
<1, 2, 3>
<1, 2, 3>
<1, 2, 3>
<0, 1, 4>
<1, 2, 3>
<x, y, z>


### 3.6 Здесь нечего возвращать
Если в функции не указано возвращаемое значение, то она возвращает None. Возвращать None явным или неявным образом, решается стилистически


In [32]:
def foo1(value):
    if value:
        return value
    else:
        return None
    
def foo2(value):
    # пустая инструкция return возвращает None
    if value:
        return value
    else:
        return 
    
def foo3(value):
    # Пропущенная инструкция return возвращает None
    if value:
        return value 
    
type(foo1(0))
type(foo2(0))
type(foo3(0))  # все три функции возвращают None

NoneType